In [2]:
#PreProcessing
import pickle
import sys
#this function remove stop word from text
def Remove_Stopwords(text_tokenized):
    text_without_sw=[token for token in text_tokenized if token.lower() not in stop_words]
    return text_without_sw

#this function Remove_Punctuation from text
def Remove_Punctuation(text_tokenized_wsw):
    text_without_pun=[token for token in text_tokenized_wsw if token not in punctuations]
    return text_without_pun

#this Function coverts letters into lower.
def casefolding(text):
    text_casefolded=[]
    for token in text:
        text_casefolded.append(token.upper())
    return text_casefolded


#this word tokenize the text into words and replace - with space
def word_tokenization(data):
    data=data.lower()
    characters="abcdefghijklmnopqrstuvwxyz"
    special_character=['\n',' ']
    for i in data:
        if i == '-':
            data=data.replace(i," ")
        if i not in characters:
            if i not in special_character:
                data=data.replace(i,"")
    data=data.split()
    return(data)


#this function stem the words by porter stemmer
def stemming_words(text):
    stemmed_words=[]
    for token in text:
        stemmed_words.append(porter.stem(token))
    return stemmed_words
#finds the positions of token
def find_postions_of_token(text_tokenized, token):
    return [position for position, term in enumerate(text_tokenized) if term == token]

#check if position exist
def check_position_exist(result,positions):
    for pos in positions:
        if pos  in result:
            return 1
    return 0
#write positional_index to file
def Write_Positional_Index_to_file(Positional_index):
    sorted_Positional_index = {}
    sorted_Positional_index_keys = sorted(Positional_index.keys())  

    for term in sorted_Positional_index_keys:
        sorted_Positional_index[term] = Positional_index[term]
    file = open("Positional_index.pkl", "wb")
    pickle.dump(sorted_Positional_index, file)
    file.close()
#check validity of Query
def check_validity_of_Query(Query):
    flag=0
    for i in Query:
          if i == '\\' :
            Query=Query.replace(i," ")
            flag=1
    Query=Query.split()
    if len(Query)==3 and flag==1:
        if int(Query[2])>0:
            return 1
        else:
            return 0
    else:
        return 0
    
    
    

In [3]:
import pprint
import nltk
from nltk.tokenize import word_tokenize 
from nltk.stem import PorterStemmer
porter = PorterStemmer()

file=open("C:\\Users\\Yougesh\\CS317-IR Dataset for A1\\Stopword-List.txt",'r')

stop_words=file.read()

punctuations = '''!()-[]{};:'‘"\,<>./?@#$%^&*“’”_~'''


Positional_Index=dict()

list_t=""

for x in range(1,51):
    
    
    file=open("C:\\Users\\Yougesh\\CS317-IR Dataset for A1\\ShortStories\\"+str(x)+".txt",'r',encoding="utf8")
    
    data=file.read()
    
    text_without_sw=Remove_Stopwords(word_tokenization(data))
    
    text_cleaned=Remove_Punctuation(text_without_sw)
    
    stemmed_words=stemming_words(text_cleaned)
    
    text_casefolded=casefolding(stemmed_words)
    #Building positional_index
    if x==1:
        for token in stemmed_words:
            Positional_Index[token]={}
            Positional_Index[token][x]=find_postions_of_token(stemmed_words,token)
            
    else:
        list_keys=Positional_Index.keys()
        for token in stemmed_words:
            if token not in list_keys:
                Positional_Index[token]={}
        for token in stemmed_words:
                if token in list_keys:
                    if x not in Positional_Index.get(token):
                        Positional_Index[token][x]=find_postions_of_token(stemmed_words,token)
Write_Positional_Index_to_file(Positional_Index)

In [ ]:
#loading positional_index file to dictionary object.
file = open("Positional_index.pkl", "rb")
Positional_Index = pickle.load(file)
Query=input("Enter the Query:")
final_res_doc=[]
final_res_set=[]
res1=dict()
res2=dict()

res3=[]

res4=[]
#check validity of the Query
if check_validity_of_Query(Query)==0:
    print("Invalid Query Syntax")
    sys.exit()

for i in Query:
          if i == '\\' :
            Query=Query.replace(i," ")
Query=Query.split()



    
term1=porter.stem(Query[0].lower())
term2=porter.stem(Query[1].lower())
k=int(Query[2])
#if term1 exist in to positional_index
if term1 in Positional_Index:
    res1=Positional_Index.get(porter.stem(term1))
#if term2 exist in to positional_index
if term2 in Positional_Index:
    res2=Positional_Index.get(porter.stem(term2))


positions=[]


final_res_doc=[term for term in res1.keys() if term in res2.keys()]

for docID in final_res_doc:
    res3=res1.get(docID)
    res4=res2.get(docID)
    for loc in res3:
        for i in range(1,k+1):
            positions.append(int(loc)+int(i))
            
            if check_position_exist(res4,positions)==1:
                if docID not in final_res_set:
                    final_res_set.append(docID)
            positions.clear()

print("Result-Set:"+str(final_res_set))